# Подготовка
 1. Самостоятельно выбранными средствами (opencv, pillow (PIL), …) сгенерировать по 820 картинок размером 100х100 пикселей (px) для каждой из цифр: 0, 1, 3, 8 следующим образом (800 – тренировочная выборка, 20 – тестовая выборка № 1):
    • фон картинки белый,
    • цифра: ширина – 20 px, высота – 50 px, цвет линии – черный, цифра целиком помещается в картинку, цифра находится в случайном месте на картинке, 
    • на изображении цифра расположена так, что ее вертикальная ось параллельна оси ординат (вертикальное положение) или оси абсцисс (горизонтальное положение),
    • тренировочная выборка содержит 400 изображений каждой цифры в горизонтальном положении и 400 изображений каждой цифры в вертикальном положении,
    • тестовая выборка содержит 10 изображений каждой цифры в горизонтальном положении и 10 изображений каждой цифры в вертикальном положении,

In [ ]:
import numpy as np
import math
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import os
import shutil
import tensorflow

In [ ]:
import sys
print(sys.executable) 
import tensorflow

/usr/bin/python3


In [ ]:
def print_img(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# Количество изображений для обучения
nb_train_samples = 80
# Количество изображений для проверки
nb_validation_samples = 20
# Количество изображений для тестирования
nb_test_samples = 20
n = nb_train_samples  + nb_validation_samples+nb_test_samples

In [ ]:
config = {
    'thickness': 1,
    'data_dir': r'./data',
    'val_dir': r'./data/val',
    'train_dir': r'./data/train',
    'test_dir1': r'./data/test1',
    'test_dir2': r'./data/test2',
    'test_dir3': r'./data/test3',
    'test_dir4': r'./data/test4',
    'test_dir5': r'./data/test5',
    'nb_images': n,
}

In [ ]:
def draw0(dc, thickness):
    cv2.ellipse(dc, ((10, 30), (19, 59), 0), (0, 0, 0), thickness)


def draw1(dc, thickness):
    cv2.line(dc, (0, 20), (19, 0), (0, 0, 0), thickness)
    cv2.line(dc, (19, 0), (19, 59), (0, 0, 0), thickness)


def draw3(dc, thickness):
    cv2.ellipse(dc, (10, 22), (9, 9), 210, 0, 255, (0, 0, 0), thickness)
    cv2.ellipse(dc, (10, 40), (9, 9), 255, 0, 255, (0, 0, 0), thickness)


def draw8(dc, thickness):
    cv2.ellipse(dc, (10, 22), (9, 9), 0, 0, 360, (0, 0, 0), thickness)
    cv2.ellipse(dc, (10, 40), (9, 9), 0, 0, 360, (0, 0, 0), thickness)


In [ ]:
def create_digits(thickness):
    _dc = {
        0: np.ones((60, 20, 3), np.uint8) * 255,
        1: np.ones((60, 20, 3), np.uint8) * 255,
        3: np.ones((60, 20, 3), np.uint8) * 255,
        8: np.ones((60, 20, 3), np.uint8) * 255,
    }
    draw0(_dc[0], thickness)
    draw1(_dc[1], thickness)
    draw3(_dc[3], thickness)
    draw8(_dc[8], thickness)
    return _dc


In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "0"))
    os.makedirs(os.path.join(dir_name, "1"))
    os.makedirs(os.path.join(dir_name, "3"))
    os.makedirs(os.path.join(dir_name, "8"))


In [ ]:
def rand_point():
    return np.random.randint(0, 79), np.random.randint(0, 39)


def generate_dataset(d, _dc, _config):
    for i in range(_config['nb_images']):
        _canvas = np.ones((100, 100, 3), np.uint8) * 255
        _x, _y = rand_point()
        _canvas[_y:_y + _dc[d].shape[0], _x:_x + _dc[d].shape[1]] = _dc[d]
        if (i >= nb_train_samples//2  and i < nb_train_samples) or (i >= nb_validation_samples//2 + nb_train_samples and i <nb_validation_samples + nb_train_samples)  or  i >= nb_train_samples +  nb_validation_samples  + nb_test_samples //2:
            _canvas = np.rot90(_canvas)
     #   if i == 501:
      #      print_img(_canvas)
        cv2.imwrite(os.path.join(config['data_dir'], str(d) + '_' + str(i) + ".jpg"), _canvas)

In [ ]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "0_" + str(i) + ".jpg"),os.path.join(dest_dir, "0"))
        shutil.copy2(os.path.join(source_dir, "1_" + str(i) + ".jpg"),os.path.join(dest_dir, "1"))
        shutil.copy2(os.path.join(source_dir, "3_" + str(i) + ".jpg"),os.path.join(dest_dir, "3"))
        shutil.copy2(os.path.join(source_dir, "8_" + str(i) + ".jpg"),os.path.join(dest_dir, "8"))


In [ ]:
create_directory(config["train_dir"])
create_directory(config["val_dir"])
create_directory(config["test_dir1"])
create_directory(config["test_dir2"])
create_directory(config["test_dir3"])
create_directory(config["test_dir4"])
create_directory(config["test_dir5"])


In [ ]:
dc = create_digits(config['thickness'])
generate_dataset(0, dc, config)
generate_dataset(1, dc, config)
generate_dataset(3, dc, config)
generate_dataset(8, dc, config)

In [ ]:
start_val_data_idx = nb_train_samples
start_test_data_idx = nb_train_samples +  nb_validation_samples
copy_images(0, start_val_data_idx, config['data_dir'], config['train_dir'])
copy_images(start_val_data_idx, start_test_data_idx, config['data_dir'],  config['val_dir'])
copy_images(start_test_data_idx, n, config['data_dir'],  config['test_dir1'])

def pxl_gen(count, img):
    for i in range(count):
        x = np.random.randint(0, 100)
        y = np.random.randint(0, 100)
        img[x][y] = (0, 0, 0)

   2. Создать новые тестовые картинки, полученные путем добавления черных пикселей (шум) в случайно выбранные места сгенерированных тестовых картинок:
    a) 20 px (тестовая выборка № 2),
    b) 50 px (тестовая выборка № 3),
    c) 100 px (тестовая выборка № 4),
    d) 200 px (тестовая выборка № 5).

In [ ]:
for k in [0, 1, 3, 8]:
    for i in range(nb_train_samples + nb_validation_samples , n):
        fname = str(k) + "_" + str(i) + '.jpg'
        # print(os.path.join(config["test_dir1"], str(k), fname))
        img = cv2.imread(os.path.join(config["test_dir1"], str(k), fname))
        dic = {2: 20, 3: 50, 4: 100, 5: 200}
        for ntest, px in dic.items():
            img_copy = img.copy()
            pxl_gen(px, img_copy)
            test_dir = "test_dir" + str(ntest)
            cv2.imwrite(os.path.join(config[test_dir], str(k), fname), img_copy)

# Задание 
Не используя предобученные модели (сети), модифицировать скрипт задачи «Dogs vs Cats» (из презентации лекции) или написать свою нейронную сеть на keras или torch такую, что:
    1) На вход подается тренировочное множество: по 800 картинок каждой цифры.
    2) Из тренировочного множества выделяется часть картинок (10–20%), на валидационное множество, в котором должны присутствовать цифры в вертикальном и горизонтальном положении. 
    3) Протестировать адекватность модели на всех тестовых выборках № 1, № 2, № 3, № 4, № 5, фиксируя при этом точность (accuracy) классификации.
    4) Повторить пункты 1)–3), изменив объем тренировочной выборки до 600, 400, 200, 100 картинок каждой цифры.
Результаты оформить в виде таблицы со столбцами: размер тренировочной выборки, количество шумовых пикселей, точность (accuracy) классификации. Полученные наборы изображений сохранить для следующих лабораторных работ.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense


In [ ]:
data_dir = r'./data/train'
# Каталог с данными для обучения
train_dir = r'./data/train'
# Каталог с данными для проверки
val_dir = r'./data/val'
# Каталог с данными для тестирования
test_dir = r'./data/test1'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = n


In [ ]:
from time import time
t1 = time()

In [ ]:

# Размеры изображения
img_width, img_height = 100, 100
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 20
# Размер мини-выборки
batch_size = 20



In [ ]:
# Создаем сверточную нейронную сеть. Архитектура сети:
# Слой свертки, размер ядра 3х3, количество карт признаков - 32 шт., функция активации ReLU.
# Слой подвыборки, выбор максимального значения из квадрата 2х2
# Слой свертки, размер ядра 3х3, количество карт признаков - 32 шт., функция активации ReLU.
# Слой подвыборки, выбор максимального значения из квадрата 2х2
# Слой свертки, размер ядра 3х3, количество карт признаков - 64 шт., функция активации ReLU.
# Слой подвыборки, выбор максимального значения из квадрата 2х2
# Слой преобразования из двумерного в одномерное представление
# Полносвязный слой, 64 нейрона,функция активации ReLU.
# Слой Dropout.
# Выходной слой, 1 нейрон, функция активации softmax
# Слои с 1 по 6 используются для выделения важных признаков в изображении, а слои с 7 по 10 - для классификации.

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 activation_25 (Activation)  (None, 98, 98, 32)        0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 47, 47, 32)        9248      
                                                                 
 activation_26 (Activation)  (None, 47, 47, 32)        0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                 

In [ ]:
# Компилируем нейронную сеть
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
# Генератор изображений создается на основе класса ImageDataGenerator.
# Генератор делит значения всех пикселов изображения на 255.
datagen = ImageDataGenerator(rescale=1. / 255)

# Генератор данных для обучения на основе изображений из каталога
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Генератор данных для проверки на основе изображений из каталога
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')




Found 320 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [ ]:
# Обучаем модель с использованием генераторов
# train_generator - генератор данных для обучения
# validation_data - генератор данных для проверки
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


4/4 [==============================] - 2s 306ms/step - loss: 0.6469 - accuracy: 0.1875 - val_loss: 0.5789 - val_accuracy: 0.0500
Epoch 2/20
4/4 [==============================] - 1s 248ms/step - loss: 0.6024 - accuracy: 0.2125 - val_loss: 0.5881 - val_accuracy: 0.1000
Epoch 3/20
4/4 [==============================] - 1s 250ms/step - loss: 0.5922 - accuracy: 0.3000 - val_loss: 0.5606 - val_accuracy: 0.3500
Epoch 4/20
4/4 [==============================] - 1s 253ms/step - loss: 0.5816 - accuracy: 0.2625 - val_loss: 0.5688 - val_accuracy: 0.2000
Epoch 5/20
4/4 [==============================] - 1s 252ms/step - loss: 0.5766 - accuracy: 0.3375 - val_loss: 0.5587 - val_accuracy: 0.3500
Epoch 6/20
4/4 [==============================] - 1s 249ms/step - loss: 0.5813 - accuracy: 0.2500 - val_loss: 0.5672 - val_accuracy: 0.3000
Epoch 7/20
4/4 [==============================] - 1s 249ms/step - loss: 0.5833 - accuracy: 0.2500 - val_loss: 0.5712 - val_accuracy: 0.2500
Epoch 8/20
4/4 [===============

In [ ]:
def test_model(test_dir):
  test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
  print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

  

In [ ]:
test_model(r'./data/test1')
test_model("./data/test2")
test_model("./data/test3")
test_model("./data/test4")
test_model("./data/test5")

Found 80 images belonging to 4 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  import sys


Аккуратность на тестовых данных: 35.00%
Found 80 images belonging to 4 classes.
Аккуратность на тестовых данных: 35.00%
Found 80 images belonging to 4 classes.
Аккуратность на тестовых данных: 30.00%
Found 80 images belonging to 4 classes.
Аккуратность на тестовых данных: 25.00%
Found 80 images belonging to 4 classes.
Аккуратность на тестовых данных: 30.00%


In [ ]:
print(time() - t1)

25.2385196685791
